In [9]:
import requests
import re
import json

parameter = {
        "newmap": "1",
        "reqflag": "pcmap",
        "biz": "1",
        "from": "webmap",
        "da_par": "direct",
        "pcevaname": "pc4.1",
        "qt": "con",
        "c": input('城市名称：'),            # 城市代码
        "wd": input('关键词：'),       # 搜索关键词
        "wd2": "",
        "pn": int(input('页数：')),           # 页数
        "nn": 4 * 10,
        "db": "0",
        "sug": "0",
        "addr": "0",
        "da_src": "pcmappg.poi.page",
        "on_gel": "1",
        "src": "7",
        "gr": "3",
        "l": "12",
        "tn": "B_NORMAL_MAP",
        # "u_loc": "12621219.536556,2630747.285024",
        "ie": "utf-8",
        # "b": "(11845157.18,3047692.2;11922085.18,3073932.2)",  #这个应该是地理位置坐标，可以忽略
        "t": "1468896652886"
    }
url = 'http://map.baidu.com/'
htm = requests.get(url, params=parameter)
#print(json(htm.content))
htm = htm.text.encode('latin-1').decode('unicode_escape')  # 转码
pattern = r'(?<=\baddress_norm":"\[).+?(?="ty":)'
htm = re.findall(pattern, htm)   # 按段落匹配
 
for r in htm:
    pattern = r'(?<=\b"\},"name":").+?(?=")'
    name = re.findall(pattern, r)
    if not name:
        pattern = r'(?<=\b,"name":").+?(?=")'
        name = re.findall(pattern, r)
    print(name[0])   # 名称
 
    pattern = r'.+?(?=")'
    adr = re.findall(pattern, r)
    pattern = r'\(.+?\['
    address = re.sub(pattern, ' ', adr[0])
    pattern = r'\(.+?\]'
    address = re.sub(pattern, ' ', address)
    print(address)   # 地址
 
    pattern = r'(?<="phone":").+?(?=")'
    phone = re.findall(pattern, r)
    print(phone[0])   #电话



城市名称：上海
关键词：咖啡店
页数：1
星巴克(大学路店)
上海市 杨浦区 大学路 
",
",
上海市 浦东新区 博山东路 黄金钫 
(021)50387707
星巴克(国康路店)
上海市 杨浦区 国康路 
(021)55080269
西西弗书店(嘉里城广场店)
上海市 浦东新区 芳甸路 浦东嘉里城F2
(021)50836597
",
上海市 浦东新区 陆家嘴环路 恒生银行大厦 
(021)68413880
卓岛咖啡(德华路店)
上海市 嘉定区 德华路 
(021)69124717
雨萨咖啡
上海市 浦东新区 川银路 
(021)58903202
星巴克(上海浦东国际机场店)
上海市 浦东新区 迎宾大道 候机楼三层南岛区1号铺
(021)68332230
",
上海市 长宁区 天山路 虹桥天都 
(021)62332278
初心慢衍CAFE
上海市 崇明区 广福路 
13818685137
张湾村二区36号楼
北京市 丰台区 丰台南路 
",


d:\virtual\python3\lib\site-packages\ipykernel_launcher.py:35: DeprecationWarning: invalid escape sequence '\/'


In [2]:
import requests
import json
import pymysql
from datetime import datetime

#从txt文件中获取相关城市并重新生成一个列表
city_list=[]
with open('city.txt','r') as f:
    for eachline in f:
        if eachline !='' and eachline !='\n':
            city=eachline.split('\t')[0]
            city_list.append(city)
    f.close()

#定义一个getjson函数用来解析返回的数据
def getjson(palace,page_num=0):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
    }
    url='http://api.map.baidu.com/place/v2/search'
    params={
        'q':'公园',
        'region':palace,
        'scope':'2',
        'page_size':'20',
        'page_num':page_num,
        'output':'json',
        'ak':'tmdOWRA5iN76ULgPrczPYHXCS4oj4162',
    }
    response=requests.get(url=url,params=params,headers=headers)
    html=response.text
    decodejson=json.loads(html)
    return decodejson

#连接数据库、获得游标，获取数据并插入到数据库中
#在获取数据时使用get()方法比较好，避免造成无相关数据时程序的中断
conn=pymysql.connect(host='localhost',user='root',password='123456',db='baidudata',charset='utf8')
cur=conn.cursor()
cur.execute("DROP TABLE IF EXISTS baidumap_taizhou")
aql1='''CREATE TABLE baidumap_taizhou(
                    ID int primary key auto_increment not null,
                    city VARCHAR(50) not null,
                    name VARCHAR(50) not null,
                    location_lat VARCHAR(50) not null,
                    location_lng VARCHAR(50) not null,
                    address VARCHAR(50)  null,
                    street_id VARCHAR(50) null,
                    time VARCHAR(50);'''
try:
    cur.execute(aql1)
except Exception as e:
            print(e)
# for city in city_list:
#     not_last_page=True
#     page_num=0
#     while not_last_page:
#         decodejson=getjson(city,page_num)
#         print(decodejson)
#         print(city,page_num)
#         if decodejson.get('results'):
#             for result in decodejson.get('results'):
#                 park=result.get('name')
#                 lat=result.get('location').get('lat')
#                 lng=result.get('location').get('lng')
#                 address=result.get('address')
#                 street_id=result.get('street_id')
#                 uid=result.get('uid')
#                 sql="""INSERT INTO baidumap.taizhou 
# (city,name,location_lat,location_lng,address,street_id,uid,time)
# VALUES (%s,%s,%s,%s,%s,%s,%s,%s);"""%(city,park,lat,lng,address,street_id,uid,datetime.now())
#                 cur.execute(sql)
#                 conn.commit()
#             page_num=page_num+1
#         else:
#             not_last_page=False
# cur.close()
# conn.close()

FileNotFoundError: [Errno 2] No such file or directory: 'city.txt'

In [12]:
#第二种百度爬虫
from datetime import datetime
import requests
import json
import pymysql as MySQLdb

#city表中拿到uid
conn=MySQLdb.connect(host='localhost',user='root',password='123456',db='baidumap',charset='utf8')
cur=conn.cursor()
sql="Select uid from baidumap.city WHERE id>0;"
cur.execute(sql)
conn.commit()
uids=cur.fetchall()

##定义一个getjson函数用来解析返回的数据
def getjson(uid):
    try:
        headers={
            'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
        }
        params={
            'uid':uid,
            'scope':'2',
            'output':'json',
            'ak':'XM53LMurtNQaAPFuKVy1WzSyZCNmNA9H',
        }
        url='http://api.map.baidu.com/place/v2/detail'
        response=requests.get(url=url,headers=headers,params=params)
        html=response.text
        decodejson=json.loads(html)
        return decodejson
    except:
        pass

#获取数据，存储数据
for uid in uids:
    uid=uid[0]
    print(uid)
    decodejson=getjson(uid)
    data=decodejson.get('result')
    if data:
        park=data.get('name')
        location_lat = data.get('location').get('lat')
        location_lng=data.get('location').get('lng')
        address=data.get('address')
        street_id=data.get('street_id')
        telephone=data.get('telephone')
        detail=data.get('detail')
        uid=data.get('uid')
        tag=data.get('detail_info').get('tag')
        detail_url=data.get('detail_info').get('detail_url')
        type=data.get('detail_info').get('type')
        overall_rating=data.get('detail_info').get('overall_rating')
        image_num=data.get('detail_info').get('image_num')
        comment_num=data.get('detail_info').get('comment_num')
        shop_hours=data.get('detail_info').get('shop_hours')
        alias=data.get('detail_info').get('alias')
        scope_type=data.get('detail_info').get('scope_type')
        scope_grade=data.get('detail_info').get('scope_grade')
        description=data.get('detail_info').get('description')
        sql="""INSERT INTO baidumap.park(park,location_lat,location_lng,address,street_id,telephone,
detail,uid,tag,detail_url,type,overall_rating,image_num,comment_num,shop_hours,alias,scope_type,scope_grade,
description,time) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);"""
        cur.execute(sql,(park,location_lat,location_lng,address,street_id,telephone,
detail,uid,tag,detail_url,type,overall_rating,image_num,comment_num,shop_hours,alias,scope_type,scope_grade,
description,datetime.now()))
        conn.commit()
cur.close()
conn.close()


InternalError: (1049, "Unknown database 'baidumap'")

In [ ]:
import requests
import re
import json

parameter = {
        "newmap": "1",
        "reqflag": "pcmap",
        "biz": "1",
        "from": "webmap",
        "da_par": "direct",
        "pcevaname": "pc4.1",
        "qt": "con",
        "c": input('城市名称：'),            # 城市代码
        "wd": input('关键词：'),       # 搜索关键词
        "wd2": "",
        "pn": int(input('页数：')),           # 页数
        "nn": 4 * 10,
        "db": "0",
        "sug": "0",
        "addr": "0",
        "da_src": "pcmappg.poi.page",
        "on_gel": "1",
        "src": "7",
        "gr": "3",
        "l": "12",
        "tn": "B_NORMAL_MAP",
        # "u_loc": "12621219.536556,2630747.285024",
        "ie": "utf-8",
        # "b": "(11845157.18,3047692.2;11922085.18,3073932.2)",  #这个应该是地理位置坐标，可以忽略
        "t": "1468896652886"
    }
url = 'http://map.baidu.com/'
htm = requests.get(url, params=parameter)
#print(json(htm.content))
htm = htm.text.encode('latin-1').decode('unicode_escape')  # 转码
pattern = r'(?<=\baddress_norm":"\[).+?(?="ty":)'
htm = re.findall(pattern, htm)   # 按段落匹配
 
for r in htm:
    pattern = r'(?<=\b"\},"name":").+?(?=")'
    name = re.findall(pattern, r)
    if not name:
        pattern = r'(?<=\b,"name":").+?(?=")'
        name = re.findall(pattern, r)
    print(name[0])   # 名称
 
    pattern = r'.+?(?=")'
    adr = re.findall(pattern, r)
    pattern = r'\(.+?\['
    address = re.sub(pattern, ' ', adr[0])
    pattern = r'\(.+?\]'
    address = re.sub(pattern, ' ', address)
    print(address)   # 地址
 
#     pattern = r'(?<="phone":").+?(?=")'
#     phone = re.findall(pattern, r)
#     print(phone[0])   #电话
